# DATA 606 - Initial EDA - Zach Vance

In [1]:
# imports
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, DoubleType, IntegerType, DateType
from pandas_profiling import ProfileReport
import pandas as pd

In [3]:
# Create the spark session and context.
spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [4]:
# Read in the appended data
df = spark.read.csv("../appended_data/appended_data.csv", sep=",", header=True)

In [5]:
# Verify the count
df.count()

90758

In [20]:
# Cache the dataframe
dfCache = df.cache()

In [7]:
# Run the pandas profile report for initial exploration
profile = ProfileReport(dfCache.toPandas(), title="Pandas Profiling Report")
profile

KeyboardInterrupt: 

In [8]:
# Verify the schema types by printing and checking them.
dfCache.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- MLSNumber: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- DOM: string (nullable = true)
 |-- CDOM: string (nullable = true)
 |-- ListDate: string (nullable = true)
 |-- Agreement of Sale/Signed Lease Date: string (nullable = true)
 |-- OffMarketDate: string (nullable = true)
 |-- SettledDate: string (nullable = true)
 |-- OriginalPrice: string (nullable = true)
 |-- List Price: string (nullable = true)
 |-- SoldPrice: string (nullable = true)
 |-- StreetNumber: string (nullable = true)
 |-- StreetDirection: string (nullable = true)
 |-- StreetName: string (nullable = true)
 |-- UnitNumber: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- County: string (nullable = true)
 |-- MLSArea: string (nullable = true)
 |-- Township: string (nullable = true)
 |-- S

In [11]:
import pyspark.sql.functions as F

# Verify the unique 4 counties exist - Anne Arundel, Baltimore, Harford, Howard
dfCache.agg(F.countDistinct('County')).show()

+-------------+
|count(County)|
+-------------+
|            4|
+-------------+



In [26]:

# First attempt at getting the average price per date per county.

dfCache["SoldPrice"].cast(IntegerType())
dfCache = dfCache.filter(dfCache.SoldPrice.isNotNull())
dfCache = dfCache.filter(dfCache.SettledDate.isNotNull())

to_date(dfCache("SettledDate"),"MM-dd-yyyy") 
#pddf = dfCache.toPandas()
#pddf
dfCache.groupBy("SettledDate", "County").agg({'SoldPrice':'avg'}).orderBy("SettledDate").show()


NameError: name 'to_date' is not defined

In [29]:
# Use SQL to query for the average price per county, per date.  I'd like to do this for each month but have
# not quite figured out how to yet.

dfCache["SoldPrice"].cast(IntegerType())
dfCache = dfCache.filter(dfCache.SoldPrice.isNotNull())
dfCache = dfCache.filter(dfCache.SettledDate.isNotNull())
dfCache.createOrReplaceTempView('housing_data')

query = """
SELECT to_date(SettledDate, 'MM/dd/yyyy') AS Date, County, avg(SoldPrice) as Price
FROM housing_data
GROUP BY Date, County
ORDER BY Date desc
"""
output = spark.sql(query)
output.show()

# Data is shown to ensure it looks correct.

Py4JJavaError: An error occurred while calling o223.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 1 times, most recent failure: Lost task 0.0 in stage 28.0 (TID 1260) (macbook-pro-3.fios-router.home executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading of Spark 3.0: Fail to parse '6/11/21' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:150)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601Timesta